### Download data of Sourcerank components and trove classifiers

In [1]:
!python -m ipykernel install --user --name=venv

Installed kernelspec venv in /home/agustin/.local/share/jupyter/kernels/venv


In [223]:
#!pip install pypi-json
#!pip install pybraries

In [18]:
import time

import pybraries.search as pys
import pandas as pd

import src.features.build_features as bf

Read reducto table to extract package names

In [12]:
reducto_table = bf.get_reducto_reports_table()

In [81]:
sourceranks = []

#### Libraries.io download

Query the api using pybraries.

The process was run by small sets of packages, and saved to csv.


In [88]:
search = pys.Search()
platform = "pypi"


for i, pkg in enumerate(reducto_table.index[3000:]):
    t = time.time()
    try:
        print("downloading: ", pkg)
        sourceranks.append({pkg: search.project_sourcerank(platform, pkg)})
        print(f"Seconds elapsed: {time.time() - t}")
    except:
        print("failed: ", pkg)
        fails.append(pkg)
    time.sleep(1)  # Allow a second between downloads. The API limits to 60 requests per minute

downloading:  blessings
Seconds elapsed: 0.4553096294403076
downloading:  dask
Seconds elapsed: 0.40807008743286133
downloading:  statsd
Seconds elapsed: 0.5568771362304688
downloading:  blinker
Seconds elapsed: 2.1709675788879395
downloading:  decorator
Seconds elapsed: 0.22731947898864746
downloading:  idna-ssl
Seconds elapsed: 0.6326947212219238
downloading:  azure-datalake-store
Seconds elapsed: 0.6407504081726074
downloading:  pyhamcrest
Seconds elapsed: 1.0446715354919434
downloading:  hvac
Seconds elapsed: 0.22949624061584473
downloading:  maxminddb
Seconds elapsed: 0.3303215503692627
downloading:  cssselect
Seconds elapsed: 0.48478078842163086
downloading:  sphinxcontrib-applehelp
Seconds elapsed: 0.8899929523468018
downloading:  sphinxcontrib-devhelp
Seconds elapsed: 0.22354650497436523
downloading:  sphinxcontrib-htmlhelp
Seconds elapsed: 0.42778515815734863
downloading:  sphinxcontrib-qthelp
Seconds elapsed: 0.23372101783752441
downloading:  google-api-core
Seconds elapsed: 

Concat data and save

In [168]:
#pd.concat([pd.DataFrame(p, columns=list(p.keys())).T for p in sourceranks], axis=0)#.to_csv(r"sourcerank_data_renamed.csv")

#### Failed packages

Update names to match those stored in Libraries.io.

Then query again and insert to sourceranks.csv file.

In [162]:
df[df["all_prereleases"].isna()]
failed_downloads = df[df["all_prereleases"].isna()].index

# Rename packages to find them in libraries.io
renamed = []
for name in failed_downloads:
    splits = name.split("-")
    if len(splits) == 2:
        renamed.append(".".join(splits))
    elif splits[0] == "aws":
        joined1 = "-".join(splits[:2])
        joined2 = "-".join(splits[2:])
        renamed.append(".".join([joined1, joined2]))
    elif splits[0] == "twitter":
        renamed.append(".".join(splits))
    elif splits[0] == "backports":
        joined = "-".join(splits[1:])
        renamed.append(".".join([splits[0], joined]))
    elif splits[0] == "aspy":
        joined = "-".join(splits[1:])
        renamed.append(".".join([splits[0], joined]))
    elif splits[0] == "h2o":
        renamed.append("h2o-pysparkling-2.4")
    else:
        print(f"package name unidentified: {name}")


package name unidentified: ccxt


In [165]:
sourceranks = []

Extraction of initially failed packages

In [167]:
import urllib

search = pys.Search()
platform = "pypi"

renamed

for i, pkg in enumerate(renamed):
    t = time.time()
    try:
        print("downloading: ", pkg)
        sourceranks.append({pkg: search.project_sourcerank(platform, pkg)})
        print(f"Seconds elapsed: {time.time() - t}")
    except:
        try:
            if pkg == "ccxt":
                sourceranks.append({pkg: search.project_sourcerank("packagist", urllib.parse.quote_plus("ccxt/ccxt"))})
        except:
            print("failed: ", pkg)
            fails.append(pkg)
    time.sleep(1)  # Allow a second between downloads. The API limits to 60 requests per minute

downloading:  twitter.common.collections
Seconds elapsed: 0.3739893436431885
downloading:  shrub.py
Seconds elapsed: 30.20086693763733
downloading:  aws-cdk.aws-ecs
Seconds elapsed: 30.366665840148926
downloading:  aws-cdk.aws-servicediscovery
Seconds elapsed: 20.70749020576477
downloading:  aws-cdk.aws-autoscaling-hooktargets
Seconds elapsed: 14.155694007873535
downloading:  oslo.upgradecheck
Seconds elapsed: 0.43015432357788086
downloading:  aws-cdk.aws-stepfunctions
Seconds elapsed: 0.224592924118042
downloading:  oslo.reports
Seconds elapsed: 11.086191892623901
downloading:  aws-cdk.aws-codebuild
Seconds elapsed: 0.3274955749511719
downloading:  aws-cdk.aws-codecommit
Seconds elapsed: 0.32910847663879395
downloading:  oslo.versionedobjects
Seconds elapsed: 0.4326610565185547
downloading:  aws-cdk.aws-batch
Seconds elapsed: 0.22846603393554688
downloading:  zope.index
Seconds elapsed: 0.22328925132751465
downloading:  aws-cdk.aws-codepipeline
Seconds elapsed: 0.23154878616333008
dow

### Download Trove Classifiers

Extract the info using PyPIJSON api.

In [179]:
trove_classifiers = {}
fails = []


In [180]:
from pypi_json import PyPIJSON

package_names = reducto_table.index#[:2]

with PyPIJSON() as client:
    for pkg in package_names:
        t = time.time()
        try:
            print("downloading: ", pkg)
            metadata = client.get_metadata(pkg)
            print(f"Seconds elapsed: {time.time() - t}")
        except:
            print(f"package failed: {pkg}")
        trove_classifiers[pkg] = metadata.info.get("classifiers", None)
        time.sleep(.5)

downloading:  urllib3
Seconds elapsed: 0.1168663501739502
downloading:  chardet
Seconds elapsed: 0.07282114028930664
downloading:  matplotlib
Seconds elapsed: 0.21190190315246582
downloading:  schedule
Seconds elapsed: 0.17557215690612793
downloading:  tensorboardx
Seconds elapsed: 0.3323955535888672
downloading:  pep517
Seconds elapsed: 0.17700719833374023
downloading:  py-cpuinfo
Seconds elapsed: 0.262829065322876
downloading:  certbot
Seconds elapsed: 0.30371594429016113
downloading:  flask-restplus
Seconds elapsed: 0.2649831771850586
downloading:  genson
Seconds elapsed: 0.375321626663208
downloading:  soundfile
Seconds elapsed: 0.2852301597595215
downloading:  django-timezone-field
Seconds elapsed: 0.17863774299621582
downloading:  scrapy-splash
Seconds elapsed: 0.4756956100463867
downloading:  pickleshare
Seconds elapsed: 0.1538548469543457
downloading:  cachelib
Seconds elapsed: 0.295867919921875
downloading:  json-log-formatter
Seconds elapsed: 0.3044593334197998
downloading:  

In [ ]:
import json

with open("trove_classifiers.json", "w") as o:
    json.dump(trove_classifiers, o)

In [227]:
df_sourcerank = pd.read_csv("sourceranks.csv", index_col=0)
#df_sourcerank_renamed = pd.read_csv("sourcerank_data_renamed.csv", index_col=0)

#### Extract the relevant classifiers and store accordingly

In [ ]:
#trove_classifiers